In [ ]:
import pandas as pd
from sqlalchemy import create_engine
import seaborn as sns
import matplotlib.pyplot as plt


In [ ]:
file_path = r"C:\Users\aga_c\Downloads\Pliki_do_Portfolio_Lab (1)\notebooks\flight_df_01.csv"
flight_df = pd.read_csv(file_path)

print("Ramka flight_df została wczytana pomyślnie.")
print(flight_df.head()) 

In [32]:
username = 'postgres'
password = 'XXXXXXXXXXXXXXX!'
database = 'airlines'
host = 'localhost'
port = 5432  

connection_string = f"postgresql://{username}:{password}@{host}:{port}/{database}"
engine = create_engine(connection_string)

airport_list_df = pd.read_sql_table('airport_list', con=engine)

In [ ]:
duplicates_in_origin_airport_id = airport_list_df['ORIGIN_AIRPORT_ID'].duplicated().any()

if duplicates_in_origin_airport_id:
    print("W kolumnie 'origin_airport_id' występują duplikaty.")
else:
    print("W kolumnie 'origin_airport_id' nie występują duplikaty.")


In [ ]:
flight_df = flight_df.merge(
    airport_list_df[['ORIGIN_AIRPORT_ID', 'ORIGIN_CITY_NAME']],  
    how='left',  
    left_on='ORIGIN_AIRPORT_ID',  
    right_on='ORIGIN_AIRPORT_ID' 
)

print(flight_df.head())


In [38]:
flight_df = flight_df.merge(
    airport_list_df,  
    how='left', 
    left_on='DEST_AIRPORT_ID', 
    right_on='ORIGIN_AIRPORT_ID' 
)

flight_df = flight_df.rename(columns={'ORIGIN_CITY_NAME': 'DESTINATION_CITY_NAME'})

flight_df.to_csv(r"C:\Users\aga_c\Downloads\Pliki_do_Portfolio_Lab (1)\notebooks\Thisss.csv")


In [ ]:
top_airports_origin_df = (
    flight_df.groupby('ORIGIN_CITY_NAME_x')
    .size()
    .reset_index(name='flight_count')
    .sort_values(by='flight_count', ascending=False)
)

print(top_airports_origin_df.head())


In [ ]:
cleaned_flight_df = flight_df.dropna(subset=['DESTINATION_CITY_NAME'])

top_airports_destination_df = (
    cleaned_flight_df.groupby('DESTINATION_CITY_NAME')
    .size() 
    .reset_index(name='flight_count') 
    .sort_values(by='flight_count', ascending=False)
)
print(top_airports_destination_df.head())


In [9]:
username = 'postgres'
password = 'XXXXXXXXX!'
database = 'airlines'
host = 'localhost'
port = 5432  

connection_string = f"postgresql://{username}:{password}@{host}:{port}/{database}"
engine = create_engine(connection_string)

airport_weather_df = pd.read_sql_table('airport_weather', con=engine)

In [ ]:
columns_to_keep = ['STATION', 'NAME', 'DATE', 'PRCP', 'SNOW', 'SNWD', 'TMAX', 'AWND']
airport_weather_df = airport_weather_df[columns_to_keep]

print(airport_weather_df.head())


In [ ]:
airport_weather_df = airport_weather_df.merge(
    airport_list_df,  
    how='left',  
    on='NAME'    
)

print(airport_weather_df.head())

In [ ]:
airport_weather_df['DATE'] = pd.to_datetime(airport_weather_df['DATE'])

print(airport_weather_df['DATE'].dtype)


In [ ]:
flight_df['DATE'] = flight_df.apply(
    lambda row: f"{int(row['YEAR']):04d}-{int(row['MONTH']):02d}-{int(row['DAY_OF_MONTH']):02d}", axis=1
)

flight_df['DATE'] = pd.to_datetime(flight_df['DATE'])

print(flight_df[['YEAR', 'MONTH', 'DAY_OF_MONTH', 'DATE']].head())


In [18]:
flight_df = flight_df.rename(columns={'ORIGIN_AIRPORT_ID_x': 'ORIGIN_AIRPORT_ID'})

merged_df = flight_df.merge(
    airport_weather_df,
    how='left',
    on='ORIGIN_AIRPORT_ID'
)


In [ ]:
print(flight_df['TMAX'].describe())

percentiles = flight_df['TMAX'].quantile([0.01, 0.25, 0.5, 0.75, 0.99])
print("Percentyle TMAX:")
print(percentiles)

plt.figure(figsize=(10, 6))
sns.boxplot(data=flight_df, x='TMAX')
plt.title("Boxplot dla TMAX (Temperatura Maksymalna)")
plt.show()


tmax_min = flight_df['TMAX'].quantile(0.01)
tmax_max = flight_df['TMAX'].quantile(0.99)

flight_df_cleaned = flight_df[(flight_df['TMAX'] >= tmax_min) & (flight_df['TMAX'] <= tmax_max)]


flight_df_cleaned['is_delayed'] = flight_df_cleaned['DEP_DELAY'] > 15


tmax_grouped = flight_df_cleaned.groupby(flight_df_cleaned['TMAX'].round())['is_delayed'].mean().reset_index()
tmax_grouped.columns = ['TMAX', 'delay_ratio']
print(tmax_grouped.head())

plt.figure(figsize=(12, 6))
sns.lineplot(data=tmax_grouped, x='TMAX', y='delay_ratio', marker='o')
plt.title("Odsetek opóźnień lotów w zależności od temperatury maksymalnej (TMAX)")
plt.xlabel("TMAX (Temperatura Maksymalna)")
plt.ylabel("Odsetek opóźnień")
plt.grid()
plt.show()

